<a href="https://colab.research.google.com/github/Saharsh2k5/ML_ASSIGNMENT_1/blob/main/Copy_of_Task4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install scikit-learn

In [ ]:
pip install tsfel

1) Use the Decision Tree model trained on the UCI-HAR dataset to predict the activities that you performed. Report the accuracy, precision, recall and confusion matrix of the model. You have three version of UCI dataset you can use a)Raw data from accelerometer, b)TSFEL featurised data, c)Features provided by author. Choose which version to use, ensuring that your test data is similar to your training data. How did the model perform? [1 marks]

In [ ]:
!pip install langchain_groq

In [ ]:
import pandas as pd
import numpy as np
import tsfel
import random
import os
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
from langchain_groq.chat_models import ChatGroq
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings('ignore')

activity_dirs_train = {
    'LAYING': "/content/drive/MyDrive/ML Assignment1/Combined/Train/LAYING",
    'SITTING': "/content/drive/MyDrive/ML Assignment1/Combined/Train/SITTING",
    'STANDING': "/content/drive/MyDrive/ML Assignment1/Combined/Train/STANDING",
    'WALKING': "/content/drive/MyDrive/ML Assignment1/Combined/Train/WALKING",
    'WALKING_DOWNSTAIRS': "/content/drive/MyDrive/ML Assignment1/Combined/Train/WALKING_DOWNSTAIRS",
    'WALKING_UPSTAIRS': "/content/drive/MyDrive/ML Assignment1/Combined/Train/WALKING_UPSTAIRS"
}

activity_dirs_test = {
    'LAYING': "/content/drive/MyDrive/ACTIVITIES/LAYING",
    'SITTING': "/content/drive/MyDrive/ACTIVITIES/SITTING",
    'STANDING': "/content/drive/MyDrive/ACTIVITIES/STANDING",
    'WALKING': "/content/drive/MyDrive/ACTIVITIES/WALKING",
    'WALKING_DOWNSTAIRS': "/content/drive/MyDrive/ACTIVITIES/WALKING_DOWNSTAIRS",
    'WALKING_UPSTAIRS': "/content/drive/MyDrive/ACTIVITIES/WALKING_UPSTAIRS"
}

def eval(y_test, y_pred):
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred, average='weighted')
  recall = recall_score(y_test, y_pred, average='weighted')
  conf_matrix = confusion_matrix(y_test, y_pred)

  print("Accuracy=", accuracy)
  print(f"Precision: ",precision)
  print(f"Recall: ",recall)
  print("Confusion Matrix:")
  print(conf_matrix)

model_name = "llama3-70b"
groq_models = {
    "llama3-70b": "llama3-70b-8192",
    "mixtral": "mixtral-8x7b-32768",
    "gemma-7b": "gemma-7b-it",
    "llama3.1-70b": "llama-3.1-70b-versatile",
    "llama3-8b": "llama3-8b-8192",
    "llama3.1-8b": "llama-3.1-8b-instant",
    "gemma-9b": "gemma2-9b-it"
}
import json

with open('/content/drive/MyDrive/key.json') as config_file:
    config = json.load(config_file)
    Groq_Token = config['groq_token']

llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)


In [ ]:
all_a_tot = []
labels_train = []

for activity, activity_dir in activity_dirs_train.items():
    file_names = os.listdir(activity_dir)
    for file_name in file_names:
        file_path = os.path.join(activity_dir, file_name)
        data = pd.read_csv(file_path, skiprows=range(1, 500), nrows=500)
        data['a_tot'] = (data['accx']**2 + data['accy']**2 + data['accz']**2)**0.5
        all_a_tot.extend(data['a_tot'].values)
        labels_train.extend([activity] * len(data['a_tot']))

all_a_tot = np.array(all_a_tot).reshape(-1, 1)

scaler = StandardScaler()
scaler.fit(all_a_tot)

features_list_train = []
labels_train_final = []

for activity, activity_dir in activity_dirs_train.items():
    file_names = os.listdir(activity_dir)
    for file_name in file_names:
        file_path = os.path.join(activity_dir, file_name)
        data = pd.read_csv(file_path, skiprows=range(1, 200), nrows=500)
        data['a_tot'] = (data['accx']**2 + data['accy']**2 + data['accz']**2)**0.5
        data['a_tot'] = scaler.transform(data['a_tot'].values.reshape(-1, 1))

        cfg = tsfel.get_features_by_domain()
        features = tsfel.time_series_features_extractor(cfg, data['a_tot'], verbose=0)

        features_list_train.append(features.values.flatten())
        labels_train_final.append(activity)

X_train = np.array(features_list_train)
y_train = np.array(labels_train_final)

features_list_test = []
labels_test = []

for activity, activity_dir in activity_dirs_test.items():
    file_names = os.listdir(activity_dir)
    for file_name in file_names:
        file_path = os.path.join(activity_dir, file_name)
        data = pd.read_csv(file_path, skiprows=range(1, 1500), nrows=500)
        data['aT (m/s^2)'] = scaler.transform(data['aT (m/s^2)'].values.reshape(-1, 1))

        cfg = tsfel.get_features_by_domain()
        features = tsfel.time_series_features_extractor(cfg, data['aT (m/s^2)'], verbose=0)

        features_list_test.append(features.values.flatten())
        labels_test.append(activity)

features_array_test = np.array(features_list_test)
y_test = np.array(labels_test)

clf = DecisionTreeClassifier(random_state=42)

clf.fit(X_train, y_train)

y_pred = clf.predict(features_array_test)

eval(labels_test, y_pred)

Accuracy= 0.2916666666666667
Precision:  0.10555555555555556
Recall:  0.2916666666666667
Confusion Matrix:
[[0 4 0 0 0 0]
 [1 3 0 0 0 0]
 [0 3 0 1 0 0]
 [0 0 0 0 4 0]
 [0 0 0 0 4 0]
 [0 0 0 0 4 0]]


The Model performed well but needs a lot of improvement as the accuracy,precision and recall are not high. Even though the model is not predicitng the activities with higher accuracy, its able to predict the activity better than randomly guessing with an accuracy of 16.67% .

2) Use the data you collected to predict the activities that you performed. Decide whether to apply preprocessing and featurization, and if so, choose the appropriate methods. How did the model perform? [1 marks]

In [ ]:
def load_data(activity_dirs, csv_names):
    data = {}
    for activity, folder in activity_dirs.items():
        data[activity] = [pd.read_csv(f"{folder}/{csv_name}.csv") for csv_name in csv_names]
    return data

def normalize_data(df, column_name):
    scaler = StandardScaler()
    df[column_name] = scaler.fit_transform(df[[column_name]])
    return df, scaler

csv_names = ['a', 'p', 'd', 's']
data = load_data(activity_dirs_test, csv_names)

original_list = [0, 1, 2, 3]
list1 = random.sample(original_list, 2)
list2 = [num for num in original_list if num not in list1]

def process_and_normalize(activity_dirs, csv_names, indices):
    features_list = []
    labels_list = []
    scaler = None

    for activity, folder in activity_dirs.items():
        for idx in indices:
            file_path = os.path.join(folder, f"{csv_names[idx]}.csv")
            data = pd.read_csv(file_path)

            if scaler is None:
                data, scaler = normalize_data(data, 'aT (m/s^2)')
            else:
                data, _ = normalize_data(data, 'aT (m/s^2)')

            cfg = tsfel.get_features_by_domain()
            features = tsfel.time_series_features_extractor(cfg, data['aT (m/s^2)'], verbose=0)

            features_list.append(features.values.flatten())
            labels_list.extend([activity] * len(features))

    return np.array(features_list), np.array(labels_list), scaler

X_train, y_train, scaler = process_and_normalize(activity_dirs_test, csv_names, list1)
X_test, y_test, _ = process_and_normalize(activity_dirs_test, csv_names, list2)

clf = DecisionTreeClassifier(random_state=42)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

eval(y_test, y_pred)

Accuracy= 0.4166666666666667
Precision:  0.47222222222222227
Recall:  0.4166666666666667
Confusion Matrix:
[[1 1 0 0 0 0]
 [0 1 0 1 0 0]
 [0 0 1 1 0 0]
 [0 0 0 0 2 0]
 [0 0 0 0 2 0]
 [0 0 0 0 2 0]]


The results of the model's performance suggest that while the model is  effective, there is room for improvement. Using cross-validation can improve the model performance. Using more data might enhance accuracy and overall performance.

3) Use the Few-Shot prompting method using UCI-HAR dataset to predict the activities that you performed. Ensure that both your examples and test query undergo similar preprocessing. How did the model perform? [1 marks]

In [ ]:
def compute_acceleration(df, col_x='accx', col_y='accy', col_z='accz'):
    return (df[col_x]**2 + df[col_y]**2 + df[col_z]**2)**0.5

X_train = []
y_train = []

for _ in range(20):
    activity = random.choice(list(activity_dirs_train.keys()))
    activity_dir = activity_dirs_train[activity]
    csv_file = random.choice(os.listdir(activity_dir))
    file_path = os.path.join(activity_dir, csv_file)

    df = pd.read_csv(file_path)
    time_point = random.randint(0, len(df) - 1)
    acc = compute_acceleration(df.iloc[time_point])

    X_train.append(acc)
    y_train.append(activity)

X_train = np.array(X_train).reshape(-1, 1)
y_train = np.array(y_train)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

X_test = []
y_test = []

for activity, activity_dir in activity_dirs_test.items():
    for file_name in os.listdir(activity_dir):
        file_path = os.path.join(activity_dir, file_name)
        data = pd.read_csv(file_path)

        if 'aT (m/s^2)' in data.columns:
            value = data['aT (m/s^2)'].iloc[0]
            X_test.append(value)
            y_test.append(activity)

X_test_df = pd.DataFrame(X_test, columns=['aT (m/s^2)'])
y_test_df = pd.DataFrame(y_test, columns=['Activity'])

X_test_scaled = scaler.transform(X_test_df[['aT (m/s^2)']])
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=['aT (m/s^2)'])

combined_df = pd.concat([X_test_scaled_df, y_test_df], axis=1)
shuffled_df = shuffle(combined_df, random_state=42)

X_test = shuffled_df[['aT (m/s^2)']]
y_test = shuffled_df[['Activity']]

feature_label_dict = {tuple(row): label for row, label in zip(X_train, y_train)}

def classify_sample(sample, feature_label_dict):
    features = ", ".join([f"Feature_{i+1}: {value}" for i, value in enumerate(sample)])
    query = f"""
    * You are a human activities classification model.
    * Your task is to analyze the given features and classify them as one of these 6 activities: {', '.join(label_set)}.
    * Provide only the activity label.

    Examples:
    {feature_label_dict}

    Features:
    {features}
    """
    answer = llm.invoke(query)
    return answer.content.strip()

def classify_samples(X, feature_label_dict):
    predictions = [classify_sample(row, feature_label_dict) for _, row in X.iterrows()]
    return pd.Series(predictions)

label_set = list(activity_dirs_train.keys())

test_predictions = classify_samples(X_test, feature_label_dict)

test_accuracy = accuracy_score(y_test, test_predictions)
print("Few-shot Testing Accuracy: ",test_accuracy)

Few-shot Testing Accuracy:  0.16666666666666666


This low accuracy suggests that the model struggled to generalize from the few examples provided. Overall, the performance highlights a need for better example selection to achieve more accurate predictions.

4) Use the Few-Shot prompting method using the data you collected to predict the activities that you performed. Adopt proper processing methods as needed. How did the model perform? [1 marks]

In [ ]:
activity_dirs_train = {
    'LAYING': "/content/drive/MyDrive/ML Assignment1/Combined/Train/LAYING",
    'SITTING': "/content/drive/MyDrive/ML Assignment1/Combined/Train/SITTING",
    'STANDING': "/content/drive/MyDrive/ML Assignment1/Combined/Train/STANDING",
    'WALKING': "/content/drive/MyDrive/ML Assignment1/Combined/Train/WALKING",
    'WALKING_DOWNSTAIRS': "/content/drive/MyDrive/ML Assignment1/Combined/Train/WALKING_DOWNSTAIRS",
    'WALKING_UPSTAIRS': "/content/drive/MyDrive/ML Assignment1/Combined/Train/WALKING_UPSTAIRS"
}

activity_dirs_test = {
    'LAYING': "/content/drive/MyDrive/ACTIVITIES/LAYING",
    'SITTING': "/content/drive/MyDrive/ACTIVITIES/SITTING",
    'STANDING': "/content/drive/MyDrive/ACTIVITIES/STANDING",
    'WALKING': "/content/drive/MyDrive/ACTIVITIES/WALKING",
    'WALKING_DOWNSTAIRS': "/content/drive/MyDrive/ACTIVITIES/WALKING_DOWNSTAIRS",
    'WALKING_UPSTAIRS': "/content/drive/MyDrive/ACTIVITIES/WALKING_UPSTAIRS"
}

csv_names = ['a.csv', 'p.csv', 'd.csv', 's.csv']

def select_random_csv_files(available_files, num_files, seed):
    random.seed(seed)
    return random.sample(available_files, num_files)

def gather_data(activity_dirs, num_samples, chosen_files=None, test_mode=False):
    X = []
    y = []

    for activity, folder_path in activity_dirs.items():
        all_files = [f for f in os.listdir(folder_path) if f in csv_names]

        if test_mode:
            available_files = list(set(all_files) - set(chosen_files))
            # if len(available_files) != 2:
            #     raise ValueError(f"Expected 2 files left for testing in folder {folder_path}. Found {len(available_files)}.")
        else:
            available_files = all_files

        # if len(available_files) < 2:
        #     raise ValueError(f"Not enough CSV files in folder {folder_path}.")

        if not test_mode:
            chosen_files = select_random_csv_files(available_files, 2, seed=2)
        else:
            chosen_files = available_files

        for _ in range(num_samples):
            selected_file = random.choice(chosen_files)
            file_path = os.path.join(folder_path, selected_file)

            df = pd.read_csv(file_path)

            # if 'aT (m/s^2)' not in df.columns:
            #     raise ValueError(f"'aT (m/s^2)' column not found in file {file_path}.")

            # if len(df) == 0:
            #     raise ValueError(f"No data in file {file_path}.")

            time_point = random.randint(0, len(df) - 1)
            total_acc = df.iloc[time_point]['aT (m/s^2)']

            X.append(total_acc)
            y.append(activity)

    return np.array(X).reshape(-1, 1), np.array(y), chosen_files

X_train, y_train, chosen_files = gather_data(activity_dirs_test, num_samples=10)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

X_test, y_test, _ = gather_data(activity_dirs_test, num_samples=10, chosen_files=chosen_files, test_mode=True)

X_test = scaler.transform(X_test)

feature_label_dict = {tuple(row): label for row, label in zip(X_train, y_train)}

model_name = "llama3-8b"
groq_models = {
    "llama3-70b": "llama3-70b-8192",
    "mixtral": "mixtral-8x7b-32768",
    "gemma-7b": "gemma-7b-it",
    "llama3.1-70b": "llama-3.1-70b-versatile",
    "llama3-8b": "llama3-8b-8192",
    "llama3.1-8b": "llama-3.1-8b-instant",
    "gemma-9b": "gemma2-9b-it"
}
import json

with open('/content/drive/MyDrive/key.json') as config_file:
    config = json.load(config_file)
    Groq_Token = config['groq_token']

llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)

def classify_sample(sample, feature_label_dict):
    features = ", ".join([f"Feature_{i+1}: {value}" for i, value in enumerate(sample)])
    query = f"""
    * You are a human activities classification model.
    * Your task is to analyze the given features and classify them as one of these 6 activities: {', '.join(label_set)}.
    * Provide only the activity label.

    Examples:
    {feature_label_dict}

    Features:
    {features}
    """
    answer = llm.invoke(query)
    return answer.content.strip()

# Function to classify all samples
def classify_samples(X, feature_label_dict):
    predictions = [classify_sample(row, feature_label_dict) for row in X]
    return pd.Series(predictions)


test_predictions = classify_samples(X_test, feature_label_dict)

test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Few-shot Testing Accuracy: {test_accuracy:.4f}")



Few-shot Testing Accuracy: 0.2667


The Model is not good as the model is only able to predict 25% of the timepoints correctly. Further tuning, additional examples could potentially improve performance.
